In [ ]:
!pip install mtcnn
import numpy as np
%pylab inline
import matplotlib.pyplot as plt
from scipy import signal
import os
import io
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras import models
from keras.utils.np_utils import to_categorical
import skimage.draw
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.applications import vgg16
from keras.applications.vgg16 import preprocess_input
from os import listdir
from PIL import Image
from numpy import asarray
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN
from os.path import isdir
from numpy import savez_compressed
from numpy import asarray

from numpy.random import seed 
seed(42)# keras seed fixing 
import tensorflow as tf 
tf.random.set_seed(42)# tensorflow seed fixing

In [ ]:
import keras
keras.__version__

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import patches

def show_detected_face(result, detected, title="Face image"):
    plt.figure(figsize=(12, 10))
    plt.imshow(result)
    img_desc = plt.gca()
    plt.set_cmap('gray')
    plt.title(title)
    plt.axis('off')
    if len(detected) > 0:
      for patch in detected:
          
          img_desc.add_patch(
              patches.Rectangle(
                  (patch['c'], patch['r']),
                  patch['width'],
                  patch['height'],
                  fill=False,
                  color='r',
                  linewidth=2)
          )
    plt.show()

In [ ]:
from skimage import data, io
from skimage.feature import Cascade


# Load the trained file from the module root.
trained_file = data.lbp_frontal_face_cascade_filename()

# Initialize the detector cascade.
detector = Cascade(trained_file)

face_image = io.imread('Nikhil.jpg')

detected = detector.detect_multi_scale(img=face_image,
                                       scale_factor=1.2,
                                       step_ratio=1,
                                       min_size=(80, 80),
                                       max_size=(1000, 1000))

show_detected_face(face_image, detected)

In [ ]:
def crop_face(result, detected, title="Detected face"):
  if len(detected) >0:
      rostro= result[detected[0]['r']:detected[0]['r']+detected[0]['width'], detected[0]['c']:detected[0]['c']+detected[0]['height']]
      plt.figure(figsize=(8, 6))
      plt.imshow(rostro)    
      plt.title(title)
      plt.axis('off')
      plt.show()

print(detected)
crop_face(face_image, detected)

In [ ]:
# extract a single face from a given photograph
def extract_face(filename, required_size=(160, 160)):
  # load image from file
  image = Image.open(filename)
  # convert to RGB, if needed
  image = image.convert('RGB')
  # convert to array
  pixels = asarray(image)
  # create the detector, using default weights
  detector = MTCNN()
  # detect faces in the image
  results = detector.detect_faces(pixels)
  # extract the bounding box from the first face
  try:
    x1, y1, width, height = results[0]['box']
    # bug fix
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array
  except IndexError:
    face_array=[]
    print("Out of index")

# specify folder to plot
i = 1
folder = 'C:/Users/Kavya/Downloads/Whole dataset-20220413T143752Z-001/Whole dataset/Kavya/'
# enumerate files
for filename in listdir(folder):
  # path
  path = folder + filename
  # get face
  face = extract_face(path)
  from matplotlib.pyplot import figure
  figure(num=None, figsize=(12, 10))
  if face is not None:
    print(i, face.shape)
    # plot
    pyplot.subplot(4, 2, i)
    pyplot.title(filename)
    pyplot.axis('off')
    pyplot.imshow(face)
    i += 1
pyplot.show()

In [ ]:
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN


# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
  # load image from file
  image = Image.open(filename)
  # convert to RGB, if needed
  image = image.convert('RGB')
  # convert to array
  pixels = asarray(image)
  # create the detector, using default weights
  detector = MTCNN()
  # detect faces in the image
  results = detector.detect_faces(pixels)
  # extract the bounding box from the first face
  try:
    x1, y1, width, height = results[0]['box']
    # bug fix
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array
  except IndexError:
    face_array=[]
    print("Out of index")
  
 
# load images and extract faces for all images in a directory
def load_faces(directory):
  faces = list()
  # enumerate files
  for filename in listdir(directory):
    # path
    path = directory + filename
    # get face
    face = extract_face(path)
    if face is not None:
      # store
      faces.append(face)
  return faces
 
# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
	X, y = list(), list()
	# enumerate folders, on per class
	for subdir in listdir(directory):
		# path
		path = directory + subdir + '/'
		# skip any files that might be in the dir
		if not isdir(path):
			continue
		# load all faces in the subdirectory
		faces = load_faces(path)
		# create labels
		labels = [subdir for _ in range(len(faces))]
		# summarize progress
		print('>loaded %d examples for class: %s' % (len(faces), subdir))
		# store
		X.extend(faces)
		y.extend(labels)
	return asarray(X), asarray(y)

In [ ]:
# extract all faces from a given photograph
def extract_all_faces(filename, required_size=(224, 224)):
  # load image from file
  original_image = Image.open(filename)
  # convert to RGB, if needed
  image = original_image.convert('RGB')
  # convert to array
  pixels = asarray(image)
  # create the detector, using default weights
  detector = MTCNN()
  # detect faces in the image
  results = detector.detect_faces(pixels)
  # extract the bounding box from the faces
  face_array=[]
  try:
    for result in results:
      x1, y1, width, height = result['box']
      x1, y1 = abs(x1), abs(y1)
      x2, y2 = x1 + width, y1 + height
      # extract the face
      face = pixels[y1:y2, x1:x2]
      #print()
      #print(original_image)
      #print(face.shape[0])
      #print("original_image.size[0]/20")
      #print(original_image.size[0]/20)
      if face.shape[0] >= original_image.size[0]/20:
        print(face.shape)
        # resize pixels to the model size
        image = Image.fromarray(face)
        image = image.resize(required_size)
        face_array.append(asarray(image))

    return face_array
  except IndexError:
    print("Out of index")

In [ ]:
# Generate face training dataset from file folders

trainX, trainy = load_dataset('Whole dataset/')
print(trainX.shape, trainy.shape)
# load test dataset
# save arrays to one file in compressed format
savez_compressed('photos.npz', trainX, trainy)
x_train_cam, y_train_cam = data['arr_0'], data['arr_1']    #m
print('Loaded: ', x_train_cam.shape, y_train_cam.shape)    #m

In [ ]:
trainX.shape

In [ ]:
trainy[1]

In [ ]:
# Generate face training dataset from file folders

trainX, trainy = load_dataset('Kavya dataset')
print(trainX.shape, trainy.shape)
# load test dataset
# save arrays to one file in compressed format
#savez_compressed('Kavya1.npz', trainX, trainy)
x_train_kavya, y_train_kavya = data['arr_0'], data['arr_1']
print('Loaded: ', x_train_kavya.shape, y_train_kavya.shape)

In [ ]:
trainX.shape

In [ ]:
trainy[1]

In [ ]:
# Generate face training dataset from file folders

trainX, trainy = load_dataset('Nikhil V Gopal dataset')
print(trainX.shape, trainy.shape)
# load test dataset
# save arrays to one file in compressed format
savez_compressed('Nikhil.npz',trainX, trainy)
x_train_nikhil, y_train_nikhil = data['arr_0'], data['arr_1']
print('Loaded: ', x_train_nikhil.shape, y_train_nikhil.shape)

In [ ]:
trainX.shape

In [ ]:
trainy[1]

In [ ]:
# Generate face training dataset from file folders

trainX, trainy = load_dataset('Nikhil M Nair dataset')
print(trainX.shape, trainy.shape)
# load test dataset
# save arrays to one file in compressed format
savez_compressed('NikhilM.npz',trainX, trainy)
x_train_nikhilm, y_train_nikhilm = data['arr_0'], data['arr_1']
print('Loaded: ', x_train_nikhilm.shape, y_train_nikhilm.shape)

In [ ]:
trainX.shape

In [ ]:
trainy[1]

In [ ]:
# Generate face training dataset from file folders

trainX, trainy = load_dataset('Nithya dataset')
print(trainX.shape, trainy.shape)
# load test dataset
# save arrays to one file in compressed format
savez_compressed('Nithya.npz',trainX, trainy)
x_train_nithya, y_train_nithya = data['arr_0'], data['arr_1']
print('Loaded: ', x_train_nithya.shape, y_train_nithya.shape)

In [ ]:
trainX.shape

In [ ]:
trainy[1]

In [ ]:
# load the faces dataset
data = load('Nikhil.npz')     
x_train_nikhil, y_train_nikhil = data['arr_0'], data['arr_1']
print('Loaded: ', x_train_nikhil.shape, y_train_nikhil.shape)

In [ ]:
# load the faces dataset from npz files
data = load('photos.npz')     
x_train_cam, y_train_cam = data['arr_0'], data['arr_1']
print('Loaded: ', x_train_cam.shape, y_train_cam.shape)

In [ ]:
np.unique(y_train_cam, return_counts=True)

In [ ]:
x_train = np.concatenate((x_train_cam, x_train_nikhil), axis=0)
x_train.shape

In [ ]:
# load the faces dataset
data = load('Kavya.npz')     
x_train_kavya, y_train_kavya = data['arr_0'], data['arr_1']
print('Loaded: ', x_train_kavya.shape, y_train_kavya.shape)

In [ ]:
x_train = np.concatenate((x_train_cam, x_train_kavya), axis=0)
x_train.shape

In [ ]:
# load the faces dataset
data = load('NikhilM.npz')     
x_train_nikhilm, y_train_nikhilm = data['arr_0'], data['arr_1']
print('Loaded: ', x_train_nikhilm.shape, y_train_nikhilm.shape)

In [ ]:
x_train = np.concatenate((x_train_cam, x_train_nikhilm), axis=0)
x_train.shape

In [ ]:
# load the faces dataset
data = load('Nithya.npz')     
x_train_nithya, y_train_nithya = data['arr_0'], data['arr_1']
print('Loaded: ', x_train_nithya.shape, y_train_nithya.shape)

In [ ]:
x_train = np.concatenate((x_train_cam, x_train_nithya), axis=0)
x_train.shape

In [ ]:
plt.subplots(10,10,figsize=(15,15))
for i in range(90):
    plt.subplot(10,10,i+1)
    plt.title(i*1)
    plt.imshow(x_train_kavya[i*1])
    plt.xticks([])
    plt.yticks([])

In [ ]:
# load the faces dataset
data = load('photos.npz')     
X, Y = data['arr_0'], data['arr_1']
print('Loaded: ', X.shape, Y.shape)

In [ ]:
img = X[100]

In [ ]:
import cv2
import random
import numpy as np

def brightness(img, low, high):
    value = random.uniform(low, high)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype = np.float64)
    hsv[:,:,1] = hsv[:,:,1]*value
    hsv[:,:,1][hsv[:,:,1]>255]  = 255
    hsv[:,:,2] = hsv[:,:,2]*value 
    hsv[:,:,2][hsv[:,:,2]>255]  = 255
    hsv = np.array(hsv, dtype = np.uint8)
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return img

In [ ]:
img_result = brightness(img, 0.1, 2)
plt.imshow(img_result)

In [ ]:
plt.title("Original")
plt.imshow(img)

In [ ]:
import cv2
import random

def augment_data(imgs, scales, rotations, brightness_values, shifts, labels):
  timg=[]
  y_hat=[]
  for (img,label) in zip(imgs,labels):
    rows, cols, ch = img.shape 
    for j in rotations:
      f = random.uniform(0, 1)
      #f = random.choice([0,1])
      rot_mat = cv2.getRotationMatrix2D( (14,14), j, 1);
      affined_img = cv2.warpAffine( img, rot_mat, (cols, rows), borderValue=(1,1,1))
      resulting_img = affined_img
      if f == 1:
        resulting_img = cv2.flip(resulting_img, f)
      resulting_img = brightness(resulting_img, brightness_values[0], brightness_values[1])
      timg.append(resulting_img)
      y_hat.append(label)
  return np.array(timg), np.array(y_hat)

In [ ]:
scales = np.arange(0.9, 1, 0.1)
rotations = np.arange(0, 2)
brightness_values = [ 0.4, 2]
shifts = [ 0.2]
au_x, au_y = augment_data(X, scales, rotations, brightness_values, shifts, Y)

In [ ]:
au_x.shape

In [ ]:
np.unique(au_y, return_counts=True)

In [ ]:
plt.subplots(10,10,figsize=(15,15))
for i in range(100):
    plt.subplot(10,10,i+1)
    plt.imshow(au_x[i*26])
    plt.xticks([])
    plt.yticks([])

In [ ]:
from sklearn.model_selection import train_test_split
# split into train test sets
au_x_train, au_x_test, au_y_train, au_y_test = train_test_split(au_x, au_y, test_size=0.33)
print(au_x_train.shape, au_x_test.shape, au_y_train.shape, au_y_test.shape)

In [ ]:
from sklearn.preprocessing import LabelEncoder
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(au_y_train)
au_y_train = out_encoder.transform(au_y_train)
au_y_test = out_encoder.transform(au_y_test)

In [ ]:
au_y_train.shape

In [ ]:
au_y_test.shape

In [ ]:
from keras.utils import to_categorical
au_train_labels = to_categorical(au_y_train)
au_test_labels = to_categorical(au_y_test)

print(au_train_labels.shape)
print(au_test_labels.shape)

In [ ]:
savez_compressed('data_encoded.npz', au_x_train, au_train_labels, au_x_test, au_test_labels)

In [ ]:
# load the faces dataset
data = load('data_encoded.npz')     
au_x_train, au_train_labels, au_x_test, au_test_labels= data['arr_0'], data['arr_1'],data['arr_2'], data['arr_3']
print('Loaded: ', au_x_train.shape, au_train_labels.shape,  au_x_test.shape, au_test_labels.shape)

In [ ]:
from keras.layers import BatchNormalization, GlobalMaxPooling2D
from numpy.random import seed 
seed(42)# keras seed fixing 
import tensorflow as tf 
tf.random.set_seed(42)# tensorflow seed fixing

model1 = Sequential()

# Convolution layers
model1.add(Conv2D(32, (3,3), input_shape = (224, 224, 3), activation = 'relu', padding='same'))
model1.add(MaxPooling2D(pool_size=(2, 2)))

model1.add(Conv2D(64, (2, 2), activation='relu', padding='same'))
model1.add(MaxPooling2D(2))

model1.add(Conv2D(64, (2, 2), activation='relu', padding='same'))
model1.add(MaxPooling2D(pool_size=(2, 2)))

model1.add(Flatten())
model1.add(Dense(units = 64, activation = 'relu'))
model1.add(Dropout(0.2)) 

# Output
model1.add(Dense(units = 4, activation = 'softmax'))
model1.summary()

In [ ]:
# Compiling the CNN
model1.compile(optimizer = 'adam',
                   loss='categorical_crossentropy',
                   metrics = ['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(monitor='val_loss', patience = 3)
checkpoint = ModelCheckpoint('facial_recognition_MTCNN.h5py', monitor = 'val_acc')

In [ ]:
pre_au_x_train = preprocess_input(au_x_train)
pre_au_x_test = preprocess_input(au_x_test)

In [ ]:
history = model1.fit(pre_au_x_train, au_train_labels, 
                     batch_size = 128, epochs=5, 
                     validation_data = (pre_au_x_test, au_test_labels), 
                     callbacks = [early_stop, checkpoint])

In [ ]:
preds1 = model1.predict(pre_au_x_test, batch_size=50) # y_test are the truth, real labels correct ones.
# classes are the model predictions (That can be wrong or match the real ones)
print("Total predicted classes are:",len(preds1))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(au_test_labels.argmax(axis=1), preds1.argmax(axis=1))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred1 = np.argmax(preds1, axis=1)
print('Confusion Matrix')
print(confusion_matrix(au_test_labels.argmax(axis=1), y_pred1))
print('Classification Report')

target_names = ['Kavya', 'Nikhil M Nair', 'Nikhil V Gopal', 'Nithya']
print(classification_report(au_test_labels.argmax(axis=1), y_pred1, target_names=target_names))

In [ ]:
# load and evaluate a saved model
from numpy import loadtxt
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# load model
model1 = load_model('facial_recognition_MTCNN.h5py')

In [ ]:
def test_model_on_photo_all_faces_from_path(file_path,verbose=False):
  recognized_person = {0:'Kavya',1:'Nikhil M Nair', 2:'Nikhil V Gopal', 3:'Nithya'}
  plt.figure(figsize=(10, 8))
  original_image = Image.open(file_path)
  plt.title("Original photo")
  plt.imshow(original_image)
  plt.show()
  images = extract_all_faces(file_path)
  for image in images:
    preprocessed_image = preprocess_input(image)
    shaped_img = preprocessed_image.reshape(1,224,224,3)
   

    classes = model1.predict(shaped_img) # y_test are the truth, real labels correct ones.
    # classes are the model predictions (That can be wrong or match the real ones)
    print("Total predicted classes are:",len(classes))
    print(int(classes.argmax(axis=1)))
    plt.title(recognized_person[int(classes.argmax(axis=1))])
    plt.imshow(image)
    plt.show()
    if verbose:
      print(classes)

In [ ]:
test_model_on_photo_all_faces_from_path("Nikhil4.jpg")

In [ ]:
from keras.layers import BatchNormalization
from numpy.random import seed 
seed(1) # keras seed fixing import

import tensorflow
tensorflow.random.set_seed(2) # tensorflow seed fixing

model_vgg16 = vgg16.VGG16(include_top=False, pooling="max", input_shape = (224, 224, 3)) 

new_layer = Dense(256, activation = 'relu')(model_vgg16.output)
new_layer = Dropout(rate = 0.5)(new_layer)

new_layer = Dense(128, activation = 'relu')(new_layer)
new_layer = Dropout(rate = 0.5)(new_layer)

new_layer = BatchNormalization()(new_layer)
new_layer = Dense(4, activation="softmax")(new_layer)

model2 = models.Model(model_vgg16.input, new_layer)

for i in range(len(model_vgg16.layers)):
  model2.layers[i].trainable = False

print(model2.summary())

In [ ]:
# Compiling the CNN
model2.compile(optimizer = 'adam',
                   loss='categorical_crossentropy',
                   metrics = ['accuracy'])

In [ ]:
from keras.applications.vgg16 import preprocess_input
pre_au_x_train = preprocess_input(au_x_train)
pre_au_x_test = preprocess_input(au_x_test)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(monitor='val_loss', patience = 3)
checkpoint = ModelCheckpoint('facial_recongition_VGG16.h5py', monitor = 'val_accuracy', save_best_only=True)

In [ ]:
history = model2.fit(pre_au_x_train, au_train_labels, 
                     batch_size = 128, epochs=5, 
                     validation_data = (pre_au_x_test, au_test_labels), 
                     callbacks = [early_stop, checkpoint])

In [ ]:
preds = model2.predict(pre_au_x_test, batch_size=50) # y_test are the truth, real labels correct ones.
# classes are the model predictions (That can be wrong or match the real ones)
print("Total predicted classes are:",len(preds))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(au_test_labels.argmax(axis=1), preds.argmax(axis=1))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = np.argmax(preds, axis=1)
print('Confusion Matrix')
print(confusion_matrix(au_y_test, y_pred))
print('Classification Report')
target_names = ['Kavya', 'Nikhil M Nair', 'Nikhil V Gopal', 'Nithya']
print(classification_report(au_y_test, y_pred, target_names=target_names))

In [ ]:
from keras.models import load_model
best_model = load_model('facial_recongition_VGG16.h5py')

In [ ]:
def test_model_on_photo_all_faces_from_path(file_path,verbose=False):
  recognized_person = {0:'Kavya',1:'Nikhil M Nair', 2:'Nikhil V Gopal', 3:'Nithya'}
  plt.figure(figsize=(10, 8))
  original_image = Image.open(file_path)
  plt.title("Original photo")
  plt.imshow(original_image)
  plt.show()
  images = extract_all_faces(file_path)
  for image in images:
    preprocessed_image = preprocess_input(image)
    shaped_img = preprocessed_image.reshape(1,224,224,3)
    classes = model2.predict(shaped_img) # y_test are the truth, real labels correct ones.
    # classes are the model predictions (That can be wrong or match the real ones)
    print("Total predicted classes are:",len(classes))
    print(int(classes.argmax(axis=1)))
    plt.title(recognized_person[int(classes.argmax(axis=1))])
    plt.imshow(image)
    plt.show()
    if verbose:
      print(classes)

In [ ]:
test_model_on_photo_all_faces_from_path("Nikhil4.jpg")